In [54]:
import pandas as pd
import os

DATA_DIR = os.path.join("..", "data", "raw")

median_income_df = pd.read_csv(os.path.join(DATA_DIR, "Household_Income.csv"))
median_income_df.head()
cdta_df = pd.read_csv(os.path.join(DATA_DIR, "2020_Community_District_Tabulation.csv"))
cdta_df.sample(5)

,BoroCode,BoroName,CountyFIPS,CDTA2020,CDTAName,CDTAType,Shape_Length,Shape_Area,the_geom
16,3,Brooklyn,47,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,0,65273.412132,7.592072e+07,MULTIPOLYGON (((-73.96929296348496 40.70709333...
8,2,Bronx,5,BX09,BX09 Soundview-Parkchester (CD 9 Approximation),0,70725.251197,1.124081e+08,MULTIPOLYGON (((-73.83596714156843 40.81988410...
15,3,Brooklyn,47,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),0,65640.600849,1.316510e+08,MULTIPOLYGON (((-73.92405909736993 40.71411156...
12,2,Bronx,5,BX26,BX26 Van Cortlandt Park (JIA 26 Equivalent),1,32864.114365,5.065860e+07,MULTIPOLYGON (((-73.86789798562498 40.90294017...
7,2,Bronx,5,BX08,BX08 Riverdale-Kingsbridge-Marble Hill (CD 8 A...,0,50413.975891,8.626591e+07,MULTIPOLYGON (((-73.89663333863236 40.91141737...


In [55]:
# Quick check
# Check columns
print("House Hold Income Columns:\n", median_income_df.columns, "\n")
print("CDTA Columns:\n", cdta_df.columns, "\n")
median_income_df.sample(5)

House Hold Income Columns:
 Index(['Location', 'Income Level', 'TimeFrame', 'DataFormat', 'Data', 'Fips'], dtype='object') 

CDTA Columns:
 Index(['BoroCode', 'BoroName', 'CountyFIPS', 'CDTA2020', 'CDTAName',
       'CDTAType', 'Shape_Length', 'Shape_Area', 'the_geom'],
      dtype='object') 



,Location,Income Level,TimeFrame,DataFormat,Data,Fips
7894,Park Slope,"$35,000 to $49,999",2012,Percent,0.09683,306
13538,Bedford Park,"$100,000 to $199,999",2018,Number,4089.00000,207
9219,Queens Village,"$75,000 to $99,999",2013,Percent,0.15482,413
8071,Washington Heights,"$75,000 to $99,999",2012,Number,6524.00000,112
3288,Sheepshead Bay,"$15,000 to $24,999",2008,Number,6711.00000,315


In [47]:
# Get the size of the dataframe
rows, cols = cdta_df.shape
print(f"Merged DataFrame size: {rows} rows x {cols} columns")

Merged DataFrame size: 35 rows x 9 columns


In [65]:
# Step 1: Extract the code in parentheses
median_income_df['Code'] = median_income_df['Location'].str.extract(r'\((.*?)\)')
median_income_df.head()

,Location,Income Level,TimeFrame,DataFormat,Data,Fips,Code
0,Battery Park/Tribeca,"$100,000 to $199,999",2005,Number,8039.72359,101,NaN
1,Greenwich Village,"$100,000 to $199,999",2005,Number,11868.27641,102,NaN
2,Lower East Side,"$100,000 to $199,999",2005,Number,6951.00000,103,NaN
3,Chelsea/Clinton,"$100,000 to $199,999",2005,Number,10347.29257,104,NaN
4,Midtown Business District,"$100,000 to $199,999",2005,Number,5178.70743,105,NaN


In [57]:
# Step 2: Replace letters only (prefix replacements)
letter_map = {
    "K": "BK",
   
    "B": "BX",
    "M": "MN",
    
}

In [66]:

# Function to replace letters while keeping numbers
def replace_prefix(code):
    if pd.isna(code):
        return code
    for old, new in letter_map.items():
        if code.startswith(old):
            return new + code[len(old):]
    return code  # leave as-is if no match

median_income_df['Code'] = median_income_df['Code'].apply(replace_prefix)

# Optional: check first few rows
print(median_income_df.sample(5))


              Location          Income Level  TimeFrame DataFormat  \
13587  Lower East Side    $15,000 to $24,999       2018     Number   
838        Tottenville    $75,000 to $99,999       2005     Number   
4850     East Flatbush    $50,000 to $74,999       2009     Number   
9437        Mott Haven  $100,000 to $199,999       2014    Percent   
11420  Elmhurst/Corona         Under $15,000       2015    Percent   

             Data  Fips  Code  
13587  8190.00000   103   NaN  
838    9187.00000   503   NaN  
4850   9857.00000   317   NaN  
9437      0.03573   201   NaN  
11420     0.16134   404   NaN  


In [53]:

# Get the size of the dataframe
rows, cols = median_income_df.shape
print(f"Merged DataFrame size: {rows} rows x {cols} columns")

#Step 3: Export to CSV
output_fp = '/Users/danielluna/Desktop/median_income_with_codes.csv'
median_income_df.to_csv(output_fp, index=False)
print(f"Saved updated CSV to {output_fp}")
print("Median Income Columns:\n", median_income_df.columns, "\n")

# Inner join on Code -> CDTA2020
merged_df = median_income_df.merge(
    cdta_df,
    left_on='Code',
    right_on='CDTA2020',
    how='left'
)

# Sort the DataFrame by 'Code'
merged_df = merged_df.sort_values(by='Code').reset_index(drop=True)

merged_df = merged_df[['Location','All Households','Families','Families with Children','Families without Children','Code','the_geom']]

# Step 1: Strip $ and commas from income columns and convert to numeric
income_cols = ['All Households', 'Families', 'Families with Children', 'Families without Children']
for col in income_cols:
    merged_df[col] = merged_df[col].replace('[\$,]', '', regex=True).astype(float)

# Step 2: Ensure string columns
string_cols = ['Location', 'Code', 'the_geom']
for col in string_cols:
    merged_df[col] = merged_df[col].astype(str)

# Quick check
print(merged_df.head())
print(merged_df.columns)
# Get the size of the dataframe
rows, cols = merged_df.shape
print(f"Merged DataFrame size: {rows} rows x {cols} columns")

# Optional: export to CSV
output_fp = '/Users/danielluna/Desktop/median_income_cdta_merged.csv'
merged_df.to_csv(output_fp, index=False)
print(f"Saved merged CSV to {output_fp}")

Merged DataFrame size: 17680 rows x 7 columns
Saved updated CSV to /Users/danielluna/Desktop/median_income_with_codes.csv
Median Income Columns:
 Index(['Location', 'Income Level', 'TimeFrame', 'DataFormat', 'Data', 'Fips',
       'Code'],
      dtype='object') 



<>:27: SyntaxWarning: invalid escape sequence '\$'
<>:27: SyntaxWarning: invalid escape sequence '\$'
/var/folders/82/sgxft7kd6vxb73hz_4nh601w0000gn/T/ipykernel_22723/3386153353.py:27: SyntaxWarning: invalid escape sequence '\$'
  merged_df[col] = merged_df[col].replace('[\$,]', '', regex=True).astype(float)
/var/folders/82/sgxft7kd6vxb73hz_4nh601w0000gn/T/ipykernel_22723/3386153353.py:27: SyntaxWarning: invalid escape sequence '\$'
  merged_df[col] = merged_df[col].replace('[\$,]', '', regex=True).astype(float)


ValueError: You are trying to merge on float64 and object columns for key 'Code'. If you wish to proceed you should use pd.concat

In [73]:
import pandas as pd
import os

DATA_DIR = os.path.join("..", "data", "raw")


# --- File Paths ---
# NOTE: Please ensure you have these two files in the same directory as the script.
# --- Load Data ---
try:
    income_df = pd.read_csv(os.path.join(DATA_DIR, "Household_Income.csv"))

    cdta_fp = pd.read_csv(os.path.join(DATA_DIR, "2020_Community_District_Tabulation.csv"))

    print("Files loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading files: {e}")
    print("Please make sure the correct filenames are used and the files are in the right directory.")
    # Exit if files cannot be loaded.
    exit()

Files loaded successfully.


In [74]:
# --- Data Preparation ---

# 1. Prepare the income data
# The 'Fips' column seems to encode borough and district. We'll convert it to a CDTA code.
# For example, 317 -> BK17 (Brooklyn Community District 17)

# Mapping from the first digit of 'Fips' to the borough prefix
boro_map = {
    1: 'MN',  # Manhattan
    2: 'BX',  # Bronx
    3: 'BK',  # Brooklyn
    4: 'QN',  # Queens
    5: 'SI'   # Staten Island
}

def create_cdta_code(fips):
    """Converts a FIPS-like number (e.g., 317) to a CDTA code (e.g., 'BK17')."""
    if pd.isna(fips):
        return None
    fips = int(fips)
    boro_digit = fips // 100
    district_num = fips % 100

    if boro_digit not in boro_map:
        return None

    boro_prefix = boro_map[boro_digit]
    # Format number with leading zero if needed (e.g., 1 -> '01')
    return f'{boro_prefix}{district_num:02d}'

# Create the new join key in the income dataframe
income_df['CDTACode'] = income_df['Fips'].apply(create_cdta_code)
print("\nCreated 'CDTACode' in the income dataframe. Head:")
print(income_df[['Location', 'Fips', 'CDTACode']].head())


# 2. Merge the two dataframes
# We merge income data with geographic data on the new CDTA code.
merged_df = income_df.merge(
    cdta_df,
    left_on='CDTACode',
    right_on='CDTA2020',
    how='left'  # Use a left join to keep all income data
)

# Check for rows that didn't merge
unmerged_count = merged_df['the_geom'].isna().sum()
if unmerged_count > 0:
    print(f"\nWarning: {unmerged_count} rows from the income data did not find a matching geography.")

# --- Filtering ---

# 3. Filter for Bronx and Brooklyn
# We use the 'BoroName' column from the geography data to select the desired boroughs.
# We also drop any rows that didn't have a matching geometry.
bronx_brooklyn_df = merged_df[merged_df['BoroName'].isin(['Brooklyn', 'Bronx'])].dropna(subset=['the_geom']).copy()
print(f"\nFiltered data to include only Brooklyn and Bronx. Resulting shape: {bronx_brooklyn_df.shape}")


# --- Final Cleanup & Export ---

# 4. Clean up and select final columns for Kepler.gl
final_cols = [
    'Location', 'Income Level', 'TimeFrame', 'Data', 'DataFormat',
    'CDTACode', 'BoroName', 'the_geom'
]
# Filter for columns that actually exist in the dataframe
final_cols_exist = [col for col in final_cols if col in bronx_brooklyn_df.columns]
final_df = bronx_brooklyn_df[final_cols_exist]


# 5. Save the result to a new CSV file
output_fp = 'bronx_and_brooklyn_household_income.csv'
final_df.to_csv(output_fp, index=False)

print(f"\nSuccessfully saved the filtered data to '{output_fp}'")
print("\nFinal data preview:")
print(final_df.head())


Created 'CDTACode' in the income dataframe. Head:
                    Location  Fips CDTACode
0       Battery Park/Tribeca   101     MN01
1          Greenwich Village   102     MN02
2            Lower East Side   103     MN03
3            Chelsea/Clinton   104     MN04
4  Midtown Business District   105     MN05


Filtered data to include only Brooklyn and Bronx. Resulting shape: (8160, 17)

Successfully saved the filtered data to 'bronx_and_brooklyn_household_income.csv'

Final data preview:
                Location          Income Level  TimeFrame        Data  \
12            Mott Haven  $100,000 to $199,999       2005   667.08540   
13           Hunts Point  $100,000 to $199,999       2005   380.91460   
14            Morrisania  $100,000 to $199,999       2005   581.71513   
15  Concourse/Highbridge  $100,000 to $199,999       2005  1162.00000   
16    University Heights  $100,000 to $199,999       2005  1429.00000   

   DataFormat CDTACode BoroName  \
12     Number     BX01    B

In [82]:
# --- Load Data ---
try:
    poverty_df = pd.read_csv(os.path.join(DATA_DIR, "Poverty.csv"))

    cdta_fp = pd.read_csv(os.path.join(DATA_DIR, "2020_Community_District_Tabulation.csv"))

    print("Files loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading files: {e}")
    print("Please make sure the correct filenames are used and the files are in the right directory.")
    # Exit if files cannot be loaded.
    exit()

Files loaded successfully.


In [83]:
# --- Data Preparation ---

# 1. Prepare the poverty data
# Convert the 'Fips' column into a standard CDTA code (e.g., 317 -> BK17).
boro_map = {
    1: 'MN',  # Manhattan
    2: 'BX',  # Bronx
    3: 'BK',  # Brooklyn
    4: 'QN',  # Queens
    5: 'SI'   # Staten Island
}

def create_cdta_code(fips):
    """Converts a FIPS-like number (e.g., 317) to a CDTA code (e.g., 'BK17')."""
    if pd.isna(fips):
        return None
    fips = int(fips)
    boro_digit = fips // 100
    district_num = fips % 100

    if boro_digit not in boro_map:
        return None

    boro_prefix = boro_map[boro_digit]
    # Format number with a leading zero if needed (e.g., 1 -> '01')
    return f'{boro_prefix}{district_num:02d}'

# Create the new join key in the poverty dataframe
poverty_df['CDTACode'] = poverty_df['Fips'].apply(create_cdta_code)
print("\nCreated 'CDTACode' in the poverty dataframe. Head:")
print(poverty_df[['Location', 'Fips', 'CDTACode']].head())


# 2. Merge the poverty and geography dataframes
merged_df = poverty_df.merge(
    cdta_df,
    left_on='CDTACode',
    right_on='CDTA2020',
    how='left'
)

# Check for rows that didn't merge
unmerged_count = merged_df['the_geom'].isna().sum()
if unmerged_count > 0:
    print(f"\nWarning: {unmerged_count} rows from the poverty data did not find a matching geography.")


# --- Filtering ---

# 3. Filter for Bronx and Brooklyn
# We use 'BoroName' from the geography data and drop any rows without a geometry.
bronx_brooklyn_df = merged_df[merged_df['BoroName'].isin(['Brooklyn', 'Bronx'])].dropna(subset=['the_geom']).copy()
print(f"\nFiltered data to include only Brooklyn and Bronx. Resulting shape: {bronx_brooklyn_df.shape}")


# --- Final Cleanup & Export ---

# 4. Select and clean up the final columns for Kepler.gl
final_cols = [
    'Location', 'TimeFrame', 'DataFormat', 'Data',
    'CDTACode', 'BoroName', 'the_geom'
]
# Ensure all selected columns exist in the DataFrame
final_cols_exist = [col for col in final_cols if col in bronx_brooklyn_df.columns]
final_df = bronx_brooklyn_df[final_cols_exist]


# 5. Save the result to a new CSV file
output_fp = 'bronx_and_brooklyn_poverty_data.csv'
final_df.to_csv(output_fp, index=False)

print(f"\nSuccessfully saved the filtered data to '{output_fp}'")
print("\nFinal data preview:")
print(final_df.head())


Created 'CDTACode' in the poverty dataframe. Head:
                    Location  Fips CDTACode
0       Battery Park/Tribeca   101     MN01
1          Greenwich Village   102     MN02
2            Lower East Side   103     MN03
3            Chelsea/Clinton   104     MN04
4  Midtown Business District   105     MN05


Filtered data to include only Brooklyn and Bronx. Resulting shape: (1080, 16)

Successfully saved the filtered data to 'bronx_and_brooklyn_poverty_data.csv'

Final data preview:
                Location  TimeFrame DataFormat         Data CDTACode BoroName  \
12            Mott Haven       2011     Number  36723.43294     BX01    Bronx   
13           Hunts Point       2011     Number  20969.56706     BX02    Bronx   
14            Morrisania       2011     Number  35195.47786     BX03    Bronx   
15  Concourse/Highbridge       2011     Number  57290.00000     BX04    Bronx   
16    University Heights       2011     Number  52920.00000     BX05    Bronx   

                 